In [1]:
import requests
from collections import defaultdict
from datetime import datetime, timedelta


In [2]:
api_key = "c6dfc4d92a8f972d237ef696ec87b37a" 

In [3]:

def get_forecast(city):
    """Fetches 5-day weather forecast for a city using OpenWeatherMap API."""
    url_forecast = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
    response_forecast = requests.get(url_forecast)
    if response_forecast.status_code != 200:
        return "Error: Could not fetch forecast data."
    forecast_json = response_forecast.json()
    return restructure_forecast(forecast_json)

def restructure_forecast(forecast_json):
    """Restructures the forecast JSON data into a nested dictionary by date and time."""
    structured_data = defaultdict(dict)
    for entry in forecast_json['list']:
        date, time = entry['dt_txt'].split()
        structured_data[date][time] = {
            'temperature': entry['main']['temp'],
            'feels_like': entry['main']['feels_like'],
            'temp_min': entry['main']['temp_min'],
            'temp_max': entry['main']['temp_max'],
            'pressure': entry['main']['pressure'],
            'humidity': entry['main']['humidity'],
            'weather': entry['weather'][0]['description'],
            'icon': entry['weather'][0]['icon'],
            'wind_speed': entry['wind']['speed'],
            'wind_deg': entry['wind']['deg'],
            'visibility': entry['visibility'],
            'pop': entry['pop'],
            'rain': entry.get('rain', {}).get('3h', 0),
            'clouds': entry['clouds']['all']
        }
    return structured_data


In [4]:
def get_specific_forecast(forecast_data, query):
    """
    Extracts specific forecast information based on user queries.
    
    Args:
    forecast_data (dict): The structured forecast data.
    query (str): The user's query.
    
    Returns:
    str: The relevant forecast information.
    """
    today = datetime.today()
    query = query.lower()

    if 'tomorrow' in query:
        target_date = (today + timedelta(days=1)).strftime('%Y-%m-%d')
    elif 'day after tomorrow' in query:
        target_date = (today + timedelta(days=2)).strftime('%Y-%m-%d')
    else:
        # Extract specific date if mentioned in the query
        try:
            target_date = datetime.strptime(query.split()[-1], '%Y-%m-%d').strftime('%Y-%m-%d')
        except ValueError:
            # Default to today's date if no specific date is mentioned
            target_date = today.strftime('%Y-%m-%d')

    if 'morning' in query:
        target_time = '09:00:00'
    elif 'afternoon' in query:
        target_time = '15:00:00'
    elif 'evening' in query:
        target_time = '18:00:00'
    elif 'night' in query:
        target_time = '21:00:00'
    else:
        # Default to the next available time if no specific time is mentioned
        target_time = '12:00:00'

    if target_date in forecast_data:
        if target_time in forecast_data[target_date]:
            forecast = forecast_data[target_date][target_time]
            return (f"Forecast for {target_date} at {target_time}:\n"
                    f"Temperature: {forecast['temperature']}°C\n"
                    f"Feels like: {forecast['feels_like']}°C\n"
                    f"Weather: {forecast['weather']}\n"
                    f"Humidity: {forecast['humidity']}%\n"
                    f"Wind: {forecast['wind_speed']} m/s, {forecast['wind_deg']}°\n"
                    f"Visibility: {forecast['visibility']} meters\n"
                    f"Probability of precipitation: {forecast['pop']*100}%\n"
                    f"Rain: {forecast['rain']} mm\n"
                    f"Clouds: {forecast['clouds']}%")
        else:
            return f"No forecast data available for {target_date} at {target_time}."
    else:
        return f"No forecast data available for {target_date}."

# Example Usage
city = "London"
forecast_data = get_forecast(city)
if isinstance(forecast_data, dict):
    query = "What's the weather like tomorrow morning?"
    print(get_specific_forecast(forecast_data, query))
else:
    print(forecast_data)


Forecast for 2024-07-03 at 09:00:00:
Temperature: 14.97°C
Feels like: 14.35°C
Weather: light rain
Humidity: 70%
Wind: 4.82 m/s, 208°
Visibility: 10000 meters
Probability of precipitation: 22.0%
Rain: 0.12 mm
Clouds: 97%
